Import our dependencies and the Minitorch from my github

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import sys
import os
from tqdm import tqdm


from engine.minitorch import Tensor,optim

I will put a seed right now for the reproducibility of the project.

In [3]:
np.random.seed(3329)

This are the variables that we need in order to create the data folder with the dataset inside,because I can't upload the repository with the dataset of Shakespeare pre-charged.

In [4]:
data_directory = "data"
file_path = os.path.join(data_directory, "input.txt")
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

This will create a data folder if it doesn't exists

In [5]:
if not os.path.exists(data_directory):
    
    os.makedirs(data_directory)

if the inside of the data folder that we just created is empty we throw a request to the url for the dataset of Shakespeare of the github of Karpathy,raise for status to inform us what happened with the request and then open it indicating to the computer that need to transform it binary to avoid missinterpretations of characters and then finally write it down to the "input.txt" inside our data folder

In [6]:
if not os.path.exists(file_path):
    
    response = requests.get(url)
    
    response.raise_for_status()


    with open(file_path, "wb") as f:

        f.write(response.content)

        

Here what we do is open the input.txt of Shakespeare preaviously stored in the data folder and read it,the encoding utf-8 is the standard to read special characters like emojis,commas,dots,etc.\
Then the text variable is just all that concatenated ,like a list of all Shakespeare.

In [7]:
with open(file_path, 'r', encoding='utf-8') as f:

    text = f.read()

First we convert the text into a set,this will give us every character inside the text.\
Then we transform that set into a list to be able to sorted in order that the same character never change its index.\
Lastly we can see the len of the vocabulary and print it.

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)


print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


To tokenize them we just create two dictionaries, String to Integer(stoi) and Integer to String(itos).

One will have as key_value the character and as the value the index of that character in the list of chars.

And the other one will have its key_value the index and as value the character of that index in the list of chars.

This is exactly what we want because we use the list of characters and its indexes to transform them into integers and vice versa.

In [9]:
stoi = {ch:i for i,ch in enumerate(chars)}

itos = {i:ch for i,ch in enumerate(chars)}

Now here is the actual encoding and decoding.\
We gonna use lambda function because this is simple and with lambda we avoid writing a def just for this.

encoder = here we pass to lamba a string(s) then we ask for its correspondent value inside the stoi dictionary.

decoder = this will receive a list of numbers,search what character correspond to that integer inside of itos and concatenate them into a string.

In [10]:
encoder = lambda s:[stoi[c] for c in s]

decoder = lambda l: ''.join([itos[i]for i in l])